In [ ]:
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel

label_path = "/content/drive/My Drive/VisualPipeline/imagenet_classes.txt"

# candidate_captions = ['cat', 'dog', 'mouse', 'horse', 'boat']

with open(label_path, 'r') as f:
    candidate_captions = [line.strip() for line in f.readlines()]

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

image_path = "/content/drive/My Drive/VisualPipeline/cat.png"
image = Image.open(image_path)

plt.imshow(image)
plt.axis('off')
plt.show()

inputs = processor(text=candidate_captions, images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image
probs = logits_per_image.softmax(dim=1)

top_probs, top_labels = probs.topk(5)

for prob, label_idx in zip(top_probs[0], top_labels[0]):
    percentage = prob.item() * 100
    print(f"{candidate_captions[label_idx]}: {percentage:.2f}%")



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/VisualPipeline/imagenet_classes.txt'

IMAGE OBJECT DETECTION

In [ ]:
# Properly install detectron2. (Please do not install twice in both ways)
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# print("detectron2:", detectron2.__version__)

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-z0e4h3zh
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-z0e4h3zh
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0df2d73d0013db7de629602c23cc120219b4f2b8
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=5792448 sha256=c4761a3dbf3a160a8a9d8e0cd056cf

In [ ]:
!pip install torch torchvision


import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from IPython.display import HTML
from base64 import b64encode
from PIL import Image, ImageOps

def load_process_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transforms.ToTensor()(image)
    return image_tensor

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cpu'  # Use CPU
predictor = DefaultPredictor(cfg)

# Adjusted function to correctly handle image data for Detectron2
def detect_specific_object(image_tensor, object_of_interest='person'):
    image_np = image_tensor.permute(1, 2, 0).numpy()
    outputs = predictor(image_np)
    pred_classes = outputs["instances"].pred_classes.cpu().numpy()
    thing_classes = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
    pred_class_names = [thing_classes[i] for i in pred_classes]

    return any(cls_name == object_of_interest for cls_name in pred_class_names)


folder_path = '/content/drive/My Drive/VisualPipeline'

image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:

    image_tensor = load_process_image(image_file)

    image_np = (image_tensor.permute(1, 2, 0).numpy() * 255).astype("uint8")

    vis_image, predictions = detect_objects_with_Detectron2(image_np, predictor)

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
    plt.title(f'Detections for {os.path.basename(image_file)}')
    plt.axis('off')
    plt.show()



model_final_280758.pkl: 167MB [00:02, 73.7MB/s]                           


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/VisualPipeline'

VISUAL IMAGE SEARCH

In [ ]:



cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a threshold for detections
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

def detect_objects(image_path, predictor):
    image = cv2.imread(image_path)
    outputs = predictor(image)
    return outputs

def visual_search(image_dir, query, predictor):
    results = []
    for image_filename in os.listdir(image_dir):
        if not image_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        image_path = os.path.join(image_dir, image_filename)
        outputs = detect_objects(image_path, predictor)
        pred_classes = outputs["instances"].pred_classes if outputs["instances"].has("pred_classes") else []
        class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).get("thing_classes", None)
        if class_names and query in class_names:
            query_idx = class_names.index(query)
            if query_idx in pred_classes:
                results.append(image_path)
    return results

image_folder = '/content/drive/My Drive/VisualPipeline/Video'
input_string = "cat"
matching_images = visual_search(image_folder, input_string, predictor)

for image_path in matching_images:
    image = cv2.imread(image_path)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f'Matching: {os.path.basename(image_path)}')
    plt.axis('off')
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/VisualPipeline/Video'

VIDEO OBJECT DETECTOR

In [ ]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from IPython.display import HTML
from base64 import b64encode
from PIL import Image, ImageOps

def load_process_image(image_path, size=(224, 224)):
    image = Image.open(image_path).convert("RGB")
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_tensor = transforms.ToTensor()(image)
    return image_tensor

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

def detect_objects_with_Detectron2(image, predictor):
    outputs = predictor(image)
    v = Visualizer(image[:, :, ::-1],
                   metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
                   scale=1.0,
                   instance_mode=ColorMode.IMAGE_BW)
    out = v.draw_instance_predictions(outputs["instances"].to('cpu'))
    return out.get_image()[:, :, ::-1], outputs

folder_path = '/content/drive/My Drive/VisualPipeline/Video'

image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
image_files.sort()

video_path = '/content/output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_size = (224, 224)
video_writer = cv2.VideoWriter(video_path, fourcc, 20.0, video_size)

for image_file in image_files:
    image_tensor = load_process_image(image_file, size=video_size)
    image_np = (image_tensor.permute(1, 2, 0).numpy() * 255).astype("uint8")
    vis_image, predictions = detect_objects_with_Detectron2(image_np, predictor)
    vis_image_bgr = cv2.cvtColor(vis_image, cv2.COLOR_RGB2BGR)
    video_writer.write(vis_image_bgr)

video_writer.release()

mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
html = HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)
display(html)



OBject Clip FInder

In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
from PIL import Image, ImageOps
from torchvision import transforms
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

# Initialize Detectron2 with a pre-trained model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

folder_path = '/content/drive/My Drive/VisualPipeline/Video'
output_folder_path = '/content/drive/My Drive/VisualPipeline/OutputVideos'

os.makedirs(output_folder_path, exist_ok=True)

def load_process_image(image_path, size=(224, 224)):
    image = Image.open(image_path).convert("RGB")
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_tensor = transforms.ToTensor()(image)
    return image_tensor

def detect_specific_object(image_tensor, object_of_interest='person'):
    image_np = image_tensor.permute(1, 2, 0).numpy()
    outputs = predictor(image_np)
    pred_classes = outputs["instances"].pred_classes.cpu().numpy()
    thing_classes = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
    pred_class_names = [thing_classes[i] for i in pred_classes]

    return any(cls_name == object_of_interest for cls_name in pred_class_names)

image_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))])

sequences = []
current_sequence = []
for image_file in image_files:
    image_np = load_process_image(image_file)
    if detect_specific_object(image_np, object_of_interest='person'):
    else:
        if current_sequence:
            sequences.append(current_sequence)
            current_sequence = []
if current_sequence:
    sequences.append(current_sequence)

# Create a video for each sequence
for i, sequence in enumerate(sequences):
    video_path = os.path.join(output_folder_path, f'sequence_{i+1}.mp4')
    video_size = (224, 224)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(video_path, fourcc, 20.0, video_size)

    for image_file in sequence:
        image = cv2.imread(image_file)
        image_resized = cv2.resize(image, video_size)
        video_writer.write(image_resized)

    video_writer.release()

print("Video creation complete.")


Video creation complete.


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, ImageOps
from torchvision import transforms
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

def load_and_resize_image(image_path, size=(640, 480)):
    image = Image.open(image_path).convert("RGB")
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    return np.array(image)

def detect_and_visualize_keypoints(image_paths, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for image_path in image_paths:
        image_np = load_and_resize_image(image_path)
        outputs = predictor(image_np)
        v = Visualizer(image_np[:, :, ::-1],
                       MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
                       scale=1.2,
                       instance_mode=ColorMode.IMAGE_BW)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        result_image = v.get_image()[:, :, ::-1]

        cv2.imwrite(os.path.join(output_folder, os.path.basename(image_path)), result_image)

folder_path = '/content/drive/My Drive/VisualPipeline/Video'
output_folder_path = '/content/drive/My Drive/VisualPipeline/KeypointsOutput'

image_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))])

detect_and_visualize_keypoints(image_files, output_folder_path)

print("Keypoint detection and visualization complete.")




Keypoint detection and visualization complete.


KEYPOINT VIDEO

In [ ]:
import cv2
import os
import numpy as np
from PIL import Image, ImageOps
from torchvision import transforms
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from natsort import natsorted, ns

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Threshold for object detection
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cpu'  # Use CPU for inference
predictor = DefaultPredictor(cfg)

def load_and_resize_image(image_path, size=(640, 480)):
    image = Image.open(image_path).convert("RGB")
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    return np.array(image)

def detect_and_visualize_keypoints(image_paths, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for image_path in image_paths:
        image_np = load_and_resize_image(image_path)
        outputs = predictor(image_np)
        v = Visualizer(image_np[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
                       scale=1.2, instance_mode=ColorMode.IMAGE_BW)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        result_image = v.get_image()[:, :, ::-1]
        cv2.imwrite(os.path.join(output_folder, os.path.basename(image_path)), result_image)

def create_video_from_images(images_folder, output_video_path, frame_size=(640, 480), fps=20):
    image_files = [os.path.join(images_folder, img) for img in os.listdir(images_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]
    image_files = natsorted(image_files, alg=ns.IGNORECASE)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)
    for image_file in image_files:
        img = cv2.imread(image_file)
        if img is not None:
            img_resized = cv2.resize(img, frame_size)
            out.write(img_resized)
    out.release()
    print(f"Video saved to {output_video_path}")

folder_path = '/content/drive/My Drive/VisualPipeline/Video'  # Folder with original images
images_folder = '/content/drive/My Drive/VisualPipeline/KeypointsOutput'  # Folder for images with keypoints
output_video_path = '/content/drive/My Drive/VisualPipeline/KeypointsVideo.mp4'  # Output video file

image_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))])
detect_and_visualize_keypoints(image_files, images_folder)

create_video_from_images(images_folder, output_video_path)



Video saved to /content/drive/My Drive/VisualPipeline/KeypointsVideo.mp4


VIDEO ALONE

In [ ]:
import cv2
import os
from natsort import natsorted, ns

def create_video_from_images(images_folder, output_video_path, frame_size=(640, 480), fps=20):
    image_files = [os.path.join(images_folder, img) for img in os.listdir(images_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]
    image_files = natsorted(image_files, alg=ns.IGNORECASE)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

    for image_file in image_files:
        img = cv2.imread(image_file)
        if img is not None:
            # Resize image if necessary
            img_resized = cv2.resize(img, frame_size)
            out.write(img_resized)
        else:
            print(f"Warning: Could not read image file {image_file}")

    out.release()
    print(f"Video saved to {output_video_path}")

images_folder = '/content/drive/My Drive/VisualPipeline/Video'
output_video_path = 'output_video.mp4'
create_video_from_images(images_folder, output_video_path)


Video saved to output_video.mp4


INSTANCE SEGMENTATIOn

In [ ]:
import cv2
import os
import numpy as np
from PIL import Image, ImageOps
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
from natsort import natsorted, ns

def setup_cfg():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.MODEL.DEVICE = "cpu"
    return cfg

def load_and_prepare_image(image_path, size=(640, 480)):
    image = Image.open(image_path).convert("RGB")
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    return np.array(image)

def perform_instance_segmentation_and_save(image_path, cfg, output_folder):
    image = load_and_prepare_image(image_path)
    predictor = DefaultPredictor(cfg)
    outputs = predictor(image)

    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image = out.get_image()[:, :, ::-1]

    cv2.imwrite(os.path.join(output_folder, os.path.basename(image_path)), result_image)

def create_video_from_images(images_folder, output_video_path, frame_size=(640, 480), fps=20):
    image_files = [os.path.join(images_folder, img) for img in os.listdir(images_folder)]
    image_files = natsorted(image_files, alg=ns.IGNORECASE)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)
    for image_file in image_files:
        img = cv2.imread(image_file)
        img_resized = cv2.resize(img, frame_size)
        out.write(img_resized)
    out.release()

cfg = setup_cfg()

input_images_folder = '/content/drive/My Drive/VisualPipeline/Video'
processed_images_folder = '/content/drive/My Drive/VisualPipeline/InstanceSegmentationOutput'
output_video_path = '/content/drive/My Drive/VisualPipeline/InstanceSegmentationVideo.mp4'

os.makedirs(processed_images_folder, exist_ok=True)
image_files = natsorted([os.path.join(input_images_folder, f) for f in os.listdir(input_images_folder) if f.endswith(('.png', '.jpg', '.jpeg'))], alg=ns.IGNORECASE)
for image_path in image_files:
    perform_instance_segmentation_and_save(image_path, cfg, processed_images_folder)

create_video_from_images(processed_images_folder, output_video_path, frame_size=(640, 480), fps=20)

print("Instance segmentation video has been created.")


model_final_f10217.pkl: 178MB [00:00, 182MB/s]                           


Instance segmentation video has been created.
